# Settlement-level least-cost electrification (Benin)

This notebook runs the model end-to-end on `data/settlements.geojson` and produces a GeoJSON output with demand, LCOE, selected technology, and investment per settlement.

Outputs written:
- `results.geojson`



In [ ]:
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path("..").resolve()))

import geopandas as gpd
import pandas as pd

from benin_least_cost.schema import DataValidator, DataSchema as DS
from benin_least_cost.parameters import ProjectConfig
from benin_least_cost.demand import run_demand_model
from benin_least_cost.lcoe import run_lcoe_model

DATA_PATH = Path("../data/settlements.geojson")
OUT_PATH = Path("../results.geojson")

DATA_PATH.exists()


In [ ]:
gdf = gpd.read_file(DATA_PATH)

gdf = DataValidator.validate_input(gdf)

{
    "rows": len(gdf),
    "geometry_type_counts": gdf.geometry.geom_type.value_counts().to_dict(),
    "columns": len(gdf.columns),
}


## Model equations (as implemented)

### Demand

- Households:

\[
\text{households} = \left\lceil \frac{\text{population}}{\text{hh\_size}} \right\rceil
\]

- Tier assignment (RWI):

\[
\text{tier} = \begin{cases}
1 & \text{if } \text{RWI} < -0.3 \\
2 & \text{if } -0.3 \le \text{RWI} < 0.4 \\
3 & \text{if } \text{RWI} \ge 0.4
\end{cases}
\]

- Residential annual demand:

\[
E_{res} = \text{households} \cdot E_{tier}(\text{tier}) \cdot \text{uptake}
\]

- Growth multiplier over planning horizon \(H\):

\[
G = \big((1+g_{pop})(1+g_{wealth})\big)^{H}
\]

- Projected annual demand:

\[
E_{2040} = (E_{res}+E_{comm}+E_{agri}+E_{pub}) \cdot G
\]

- Peak load (using tier load factor \(LF\)):

\[
P_{peak} = \frac{E_{2040}}{8760 \cdot LF(\text{tier})}
\]

### Cost and least-cost selection

- Capital Recovery Factor (CRF):

\[
CRF(r,n)=\frac{r(1+r)^n}{(1+r)^n-1}
\]

For each technology, the model computes an annualized cost and divides by delivered energy to get LCOE (USD/kWh). The minimum LCOE is selected per settlement.



In [ ]:
config = ProjectConfig()

gdf_out = gdf.copy()
gdf_out = run_demand_model(gdf_out, config)
gdf_out = run_lcoe_model(gdf_out, config)

tech_counts = gdf_out[DS.OPTIMAL_TECH].value_counts()
total_investment = float(gdf_out[DS.INVESTMENT].sum())

tech_counts, total_investment


In [ ]:
(gdf_out[[DS.PROJECTED_DEMAND, DS.PROJECTED_PEAK, DS.LCOE_GRID, DS.LCOE_MG, DS.LCOE_SHS, DS.OPTIMAL_TECH, DS.INVESTMENT]]
 .describe(include='all')
 .T)


In [ ]:
ax = (gdf_out[DS.OPTIMAL_TECH]
      .value_counts()
      .reindex(["Grid", "MiniGrid", "SHS"], fill_value=0)
      .plot(kind="bar", title="Selected technology (count)")
     )
ax.set_xlabel("")
ax.set_ylabel("settlements")


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
for ax, col in zip(axes, [DS.LCOE_GRID, DS.LCOE_MG, DS.LCOE_SHS]):
    gdf_out[col].clip(upper=2.0).plot(kind="hist", bins=60, ax=ax, title=col)
    ax.set_xlabel("USD/kWh")

plt.tight_layout()


In [ ]:
OUT_PATH.unlink(missing_ok=True)
gdf_out.to_file(OUT_PATH, driver="GeoJSON")

OUT_PATH, OUT_PATH.exists(), OUT_PATH.stat().st_size


## Scenario: change one parameter and rerun

This section shows how to run a sensitivity scenario by changing one parameter and recomputing the outputs.



In [ ]:
config2 = ProjectConfig()
config2.planning.discount_rate = 0.12

gdf_scn = gdf.copy()
gdf_scn = run_demand_model(gdf_scn, config2)
gdf_scn = run_lcoe_model(gdf_scn, config2)

gdf_scn[DS.OPTIMAL_TECH].value_counts()
